In [22]:
# pip install --upgrade xee
import os
import numpy as np
import geopandas as gpd
import pandas as pd

# import sliderule
# from sliderule import icesat2, gedi, earthdata

import matplotlib.pyplot as plt
import seaborn as sns

import xarray as xr

import ee
import geemap
# ee.Authenticate()
# Initialize the Earth Engine API
ee.Initialize()

In [23]:
# Install this to downlaod tiles
# pip install geedim


In [24]:
#####################
## Read roi
# roi_path = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/atlantic_forest/AtlanticForest.shp' 
roi_path = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/atlantic_forest/tiles/br_af_grid90km_prj.gpkg' 
roi_gdf = gpd.read_file(roi_path)



In [25]:
roi_gdf.columns

Index(['tile_num', 'area_sq_km', 'NOME', 'SHAPE_AREA', 'SHAPE_LEN',
       'geometry'],
      dtype='object')

In [26]:
tile_num = 89
roi_gdf = roi_gdf[roi_gdf['tile_num'] == tile_num]

In [27]:
# Dissolve grid and make a 60 m buffer
roi_gdf = roi_gdf.dissolve()
roi_gdf = roi_gdf.buffer(60)
roi_gdf = gpd.GeoDataFrame(geometry=roi_gdf, crs=roi_gdf.crs)

In [28]:
# roi_gdf.plot()

In [29]:
roi_gdf_epsg4326 = roi_gdf.to_crs('EPSG:4326')


In [30]:
roi_fc = geemap.geopandas_to_ee(roi_gdf_epsg4326).first().geometry()   


In [31]:
# GEE asset
# https://brasil.mapbiomas.org/en/colecoes-mapbiomas/
EMBED_IC = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')  # 64 bands, 10 m, annual



In [32]:
# Years to filter
# years = [2020, 2022, 2024]
year = 2022

In [33]:

# -----------------------------
# 4) Build annual mosaics & download
# -----------------------------
start = f'{year}-01-01'
end   = f'{year}-12-31'

img = (EMBED_IC
       .filterDate(start, end)
       .filterBounds(roi_fc)
       .mosaic()
       .clip(roi_fc)
       .set({'year': year}))



In [34]:
# Visualize

m = geemap.Map()
m.centerObject(roi_fc, 6)             # adjust zoom as you like
m.addLayer(img)
m.addLayer(roi_fc, {}, 'ROI', False)

m

Map(center=[-24.597160160426437, -48.91946399028407], controls=(WidgetControl(options=['position', 'transparen…

In [35]:
# 3) Export settings
# -----------------------------
out_dir = f'/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/{year}'
os.makedirs(out_dir, exist_ok=True)
# out_tif = f'{out_dir}/GSE_V1_ANNUAL_{year}.tif'

scale_m = 10           # dataset resolution
target_epsg = roi_gdf.crs.to_epsg() # None
target_epsg



5880

In [36]:
# ?geemap.ee_export_image

In [37]:
# geemap.ee_export_image(
#         ee_object=img,
#         filename=out_tif,
#         scale=scale_m,
#         region=roi_fc,
#         # crs=target_crs,          
#         file_per_band=False
#     )

In [38]:
## Define grid to download tiles
fishnet_aoi = geemap.fishnet(roi_fc, rows=2, cols=2)

In [39]:
out_tif_prefix = f'{out_dir}/GSE_V1_ANNUAL_{year}_'

geemap.download_ee_image_tiles(
    img, 
    fishnet_aoi, 
    out_dir, 
    prefix=out_tif_prefix, 
    # crs="EPSG:4326", 
    crs = f'EPSG:{target_epsg}',
    scale=scale_m
)

  0%|          |0/3200 tiles [00:00<?]

  0%|          |0/3200 tiles [00:00<?]

  0%|          |0/3200 tiles [00:00<?]

  0%|          |0/3200 tiles [00:00<?]

  0%|          |0/3200 tiles [00:00<?]

  0%|          |0/3200 tiles [00:00<?]

Downloaded 6 tiles in 1426.6028962135315 seconds.


## Create VRTs

In [ ]:
# gdalbuildvrt \
#   -srcnodata 0 -vrtnodata 0 \
#   /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/VegetationMask/AtlanticForest/2024/vegmask_2024.vrt \
#   /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/VegetationMask/AtlanticForest/2024/Tiles/*.tif